# Agents and RAG, A Technical Deep Dive 

In this notebook, i'll be using the Lang and Llama family for building and exploring RAG from scratch and the techniques we can do with Agents

<img src="https://www.kdnuggets.com/wp-content/uploads/awan_getting_langchain_ecosystem_1-1024x574.png" width=700>

<img src="https://d3lkc3n5th01x7.cloudfront.net/wp-content/uploads/2023/10/12015949/LlamaIndex.png" width=700>

### Brief History

The concept of intelligent agents has evolved dramatically over the past seven decades, transforming from simple rule-based systems to today's sophisticated AI companions that can reason, plan, and act autonomously. Understanding this progression is essential because it helps us appreciate why modern agentic systems represent such a significant breakthrough and why they're becoming central to how we build AI applications. The journey began in the 1950s when researchers like Allen Newell and Herbert Simon created the Logic Theorist, a program that could prove mathematical theorems by exploring different logical paths. These early agents were like skilled craftsmen—they could perform specific tasks very well, but only within narrow, pre-defined domains. The 1970s and 1980s brought expert systems like MYCIN for medical diagnosis and DENDRAL for chemical analysis. While impressive, these systems required months of manual knowledge engineering, where human experts had to explicitly encode their domain knowledge into rigid rule sets.

The 1990s marked a shift toward more flexible software agents that could operate in networked environments and coordinate with other agents. This period introduced the concept of multi-agent systems, where multiple specialized agents could collaborate to solve complex problems. However, these systems still required extensive manual programming and could only handle situations their creators had anticipated. The real transformation began in the 2000s with machine learning advances. Agents could now learn from data rather than relying solely on hand-coded rules. Virtual assistants like Siri and Alexa brought agent technology to mainstream consumers, though they remained relatively narrow in scope—essentially sophisticated voice interfaces for search and simple task execution.

<img src="https://miro.medium.com/1*Ygen57Qiyrc8DXAFsjZLNA.gif" width=700>

The breakthrough moment arrived with large language models starting around 2020. Systems like GPT-3 and GPT-4 combined vast knowledge with sophisticated reasoning abilities, creating agents that could understand natural language, maintain context across conversations, and tackle a wide variety of tasks without task-specific programming. Unlike their predecessors, these modern agents can break down complex problems into steps, use external tools when needed, and adapt to new situations they've never encountered before. This evolution represents a fundamental shift from automation to augmentation. Where early agents automated specific, predefined tasks, today's agents can understand our goals and work as collaborative partners in problem-solving. They can handle ambiguous instructions, incomplete information, and constantly changing contexts—capabilities that make them invaluable for building sophisticated applications like retrieval-augmented generation systems.

## Agents

When we talk about agents in 2025, we're entering a landscape where the term has become both ubiquitous and somewhat ambiguous. Different organizations and researchers use "agent" to describe everything from simple chatbots to fully autonomous systems that can operate independently for weeks. This diversity in definition isn't just academic—it reflects fundamentally different architectural approaches that will determine how we build the next generation of AI applications.

<img src="https://substackcdn.com/image/fetch/$s_!A_Oy!,f_auto,q_auto:good,fl_progressive:steep/https%3A%2F%2Fsubstack-post-media.s3.amazonaws.com%2Fpublic%2Fimages%2Fc3177e12-432e-4e41-814f-6febf7a35f68_1360x972.png" width=700>

At its core, an agent is a system that can perceive its environment, make decisions, and take actions to achieve specific goals. However, the way these capabilities are implemented varies dramatically. Some define agents as fully autonomous systems that operate independently over extended periods, using various tools and adapting their strategies based on feedback. Think of these like a personal assistant who can manage your entire schedule, book flights, handle emails, and make decisions on your behalf without constant supervision.

Others use the term more broadly to describe any system that follows predefined workflows to accomplish tasks. These implementations are more like following a detailed recipe—each step is predetermined, and while the system can handle some variations, it operates within clearly defined boundaries. The distinction between these approaches is crucial because it affects everything from system reliability to development complexity.

The most useful way to think about this spectrum is through the lens of control and decision-making. Workflows are systems where large language models and tools are orchestrated through predefined code paths. Every decision point is anticipated by the developer, and the system follows predetermined logic to handle different scenarios. Agents, in contrast, are systems where the LLM dynamically directs its own processes and tool usage, maintaining control over how it accomplishes tasks. The model itself decides what to do next, which tools to use, and how to adapt when things don't go as planned.



#### Simplicity defines perfectionism not complexity




When building applications with LLMs, the fundamental principle should be finding the simplest solution that meets your requirements. This might mean not building agentic systems at all. Agentic systems inherently trade latency and cost for better task performance, and you need to carefully consider when this tradeoff makes sense for your specific use case.

When more complexity is warranted, workflows offer predictability and consistency for well-defined tasks where you can anticipate most scenarios and edge cases. They're excellent for standardized processes like data processing pipelines, content moderation, or structured analysis tasks. Agents become the better choice when you need flexibility and model-driven decision-making at scale—situations where the variety of inputs and required responses is too broad to predefine, or where the system needs to adapt to entirely new scenarios.

The reality is that for many applications, the most effective approach involves optimizing single LLM calls with retrieval and in-context examples rather than building complex agentic systems. However, as we'll explore throughout this tutorial, there are compelling scenarios where the additional complexity of agents becomes not just beneficial, but necessary for achieving your goals. Understanding when and how to make this transition is what separates effective AI system builders from those who over-engineer solutions to problems that could be solved more simply.




#### Prompts


Prompts are the fundamental interface between human intent and AI capabilities, serving as the bridge that translates our natural language requests into structured instructions that language models can understand and act upon. In the context of agentic systems, prompts become even more critical because they not only convey what we want the agent to accomplish, but also how the agent should approach problem-solving, what tools it can use, and how it should reason through complex tasks.

Think of prompts as the instruction manual for your AI agent—just as a well-written manual can make the difference between a novice successfully assembling furniture or ending up with a pile of confused parts, a well-crafted prompt determines whether your agent performs brilliantly or struggles to understand your intent. The quality and structure of your prompts directly influence the agent's reasoning capabilities, tool usage patterns, and overall effectiveness in completing tasks.

<img src="https://www.datablist.com/_next/image?url=%2Fhowto_images%2Fhow-to-write-prompt-ai-agents%2Fstructured-ai-agent-prompt.png&w=3840&q=75" width=700>

There are several types of prompts that serve different purposes in agentic systems. System prompts establish the agent's role, personality, and fundamental operating principles—these are like giving someone their job description and company handbook before they start work. User prompts contain the specific tasks or questions you want the agent to handle, while few-shot prompts provide examples of desired input-output patterns to guide the agent's responses. Chain-of-thought prompts encourage step-by-step reasoning, helping agents break down complex problems into manageable pieces.

In multi-step agentic workflows, prompt engineering becomes particularly sophisticated because you need to design prompts that not only solve individual tasks but also coordinate between different stages of processing. The agent needs to understand when to use specific tools, how to interpret tool outputs, and how to maintain context across multiple interaction cycles. This requires careful consideration of prompt structure, token efficiency, and the logical flow of information through your system.

Let's explore how to implement basic prompt templates using LangChain with Google's Gemini model to see these concepts in action:

In [ ]:
# ================================
# COMPREHENSIVE SETUP AND IMPORTS
# ================================
# This cell contains all imports and basic setup for the entire tutorial

# Core LangChain and LLM imports
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.tools import tool
from langchain.tools import Tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.chains import ConversationChain

# Memory system imports
from langchain.memory import (
    ConversationBufferMemory,
    ConversationSummaryMemory, 
    ConversationBufferWindowMemory,
    ConversationTokenBufferMemory,
    ConversationSummaryBufferMemory,
    ConversationEntityMemory,
    CombinedMemory,
    ReadOnlySharedMemory,
    SimpleMemory
)
from langchain.memory.entity import InMemoryEntityStore

# Standard library imports
import os
import json
import random
import datetime
from typing import List, Dict, Any
from dataclasses import dataclass
from abc import ABC, abstractmethod

# Mathematical libraries for calculations
import numpy as np

# ================================
# GLOBAL CONFIGURATION
# ================================

# Initialize primary LLM with balanced settings
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro", 
    temperature=0.3,  # Balanced creativity and consistency
    google_api_key=os.getenv("GOOGLE_API_KEY")
)

# Global variables for the tutorial workflow
tutorial_state = {
    "current_section": "setup",
    "demo_data": {},
    "conversation_history": [],
    "skills_registry": {},
    "memory_systems": {}
}

print("🚀 Agents and RAG Tutorial - Setup Complete")
print("📦 All imports loaded successfully")
print("🔧 Global configuration initialized")
print("📋 Tutorial state tracking ready")

In [ ]:
# ================================
# PROMPTING FUNDAMENTALS
# ================================
# Demonstrate different prompt types and their effectiveness

def create_prompt_examples():
    """Create various prompt templates for demonstration"""
    
    # Basic instructional prompt
    basic_template = PromptTemplate(
        input_variables=["topic", "audience"],
        template="""You are an expert educator who excels at explaining complex topics clearly.
        
        Topic: {topic}
        Audience: {audience}
        
        Please provide a clear, engaging explanation that includes:
        1. Core concept definition
        2. Relevant examples or analogies  
        3. Key takeaways for the audience level
        
        Keep your explanation appropriate for the specified audience."""
    )
    
    # Conversational prompt with memory
    chat_template = ChatPromptTemplate.from_messages([
        ("system", """You are a helpful AI assistant with expertise in technology and science. 
        You provide accurate, clear explanations and engage in detailed discussions.
        Always think step-by-step when solving problems and explain your reasoning."""),
        ("human", "I need help understanding {concept}. Can you break it down for me?"),
        ("ai", "I'd be happy to help explain {concept}! Let me break this down step by step."),
        ("human", "{user_question}")
    ])
    
    return basic_template, chat_template

# Create prompt templates
basic_template, chat_template = create_prompt_examples()

# Create reusable chains using LangChain Expression Language (LCEL)
basic_chain = basic_template | llm | StrOutputParser()
chat_chain = chat_template | llm | StrOutputParser()

# Store in tutorial state for later use
tutorial_state["prompt_templates"] = {
    "basic": basic_template,
    "chat": chat_template
}

tutorial_state["chains"] = {
    "basic": basic_chain,
    "chat": chat_chain
}

print("✅ Prompt Engineering Components Ready")
print("📝 Basic and conversational templates created") 
print("🔗 LCEL chains initialized and stored in tutorial state")

Great! now our LLM can respond to our questions, but how can we tweak it more to determine how much it weighs the prompt guideline while responding with it's own knowledge and reasoning? let's see!

### Hyperparameters

Hyperparameters are the control knobs that determine how a language model generates responses, acting like the settings on a sophisticated instrument that can dramatically change the output quality and behavior. Understanding these parameters is crucial for building effective agents because they directly influence how the model balances following prompt instructions versus drawing on its pre-trained knowledge, how creative or conservative its responses are, and how consistently it behaves across multiple interactions.

### Mathematical Foundation of Hyperparameters

**Temperature (τ)** controls the randomness in the model's token selection process through the softmax function. Given logits $z_i$ for each possible token $i$, the probability distribution is calculated as:

$$P(token_i) = \frac{e^{z_i/τ}}{\sum_{j=1}^{V} e^{z_j/τ}}$$

Where:
- $τ$ (tau) is the temperature parameter
- $V$ is the vocabulary size  
- Lower $τ$ → sharper distribution (more deterministic)
- Higher $τ$ → flatter distribution (more random)

At $τ = 1$, we get the standard softmax. As $τ → 0$, the distribution approaches a one-hot encoding of the highest logit. As $τ → ∞$, the distribution becomes uniform.

**Top-p (Nucleus Sampling)** works by selecting the smallest set of tokens whose cumulative probability exceeds threshold $p$:

$$\text{Nucleus} = \{i : \sum_{j \in \text{top-k tokens}} P(token_j) \leq p\}$$

**Top-k** simply restricts consideration to the $k$ highest-probability tokens, where $k$ is a fixed integer.

**Max tokens** provides an upper bound $N_{max}$ on sequence length, while **stop sequences** define termination conditions based on specific token patterns.

Let's explore how these parameters affect model behavior in practice:

In [ ]:
# ================================
# HYPERPARAMETER EXPERIMENTATION
# ================================
# Demonstrate how different hyperparameters affect model behavior

def create_hyperparameter_variants():
    """Create LLM instances with different hyperparameter settings"""
    
    # Conservative configuration (low temperature)
    conservative_llm = ChatGoogleGenerativeAI(
        model="gemini-1.5-pro",
        temperature=0.1,  # τ = 0.1 for high determinism
        max_tokens=150,
        google_api_key=os.getenv("GOOGLE_API_KEY")
    )
    
    # Balanced configuration  
    balanced_llm = ChatGoogleGenerativeAI(
        model="gemini-1.5-pro", 
        temperature=0.7,  # τ = 0.7 for creativity-consistency balance
        max_tokens=150,
        google_api_key=os.getenv("GOOGLE_API_KEY")
    )
    
    # Creative configuration (high temperature)
    creative_llm = ChatGoogleGenerativeAI(
        model="gemini-1.5-pro",
        temperature=1.2,  # τ = 1.2 for high creativity
        max_tokens=150, 
        google_api_key=os.getenv("GOOGLE_API_KEY")
    )
    
    return {
        "conservative": conservative_llm,
        "balanced": balanced_llm, 
        "creative": creative_llm
    }

def test_hyperparameter_effects(topic="quantum computing"):
    """Test how different hyperparameters affect responses"""
    
    llm_variants = create_hyperparameter_variants()
    
    # Shared prompt template
    prompt = PromptTemplate(
        input_variables=["topic"],
        template="Explain {topic} in exactly three sentences. Be accurate but engaging."
    )
    
    results = {}
    
    for config_name, llm_variant in llm_variants.items():
        chain = prompt | llm_variant | StrOutputParser()
        response = chain.invoke({"topic": topic})
        results[config_name] = response
        print(f"\n{config_name.upper()} (τ={llm_variant.temperature}):")
        print(f"Response: {response}")
    
    return results

def test_instruction_adherence():
    """Test how temperature affects prompt instruction following"""
    
    instruction_prompt = PromptTemplate(
        input_variables=["format", "content"],
        template="""You must follow this format EXACTLY: {format}
        
        Content to format: {content}
        
        CRITICAL: Strict adherence to the format is required."""
    )
    
    # High vs low temperature comparison
    strict_llm = ChatGoogleGenerativeAI(
        model="gemini-1.5-pro",
        temperature=0.0,  # Maximum determinism
        google_api_key=os.getenv("GOOGLE_API_KEY")
    )
    
    flexible_llm = ChatGoogleGenerativeAI(
        model="gemini-1.5-pro",
        temperature=0.9,  # More creativity
        google_api_key=os.getenv("GOOGLE_API_KEY") 
    )
    
    strict_chain = instruction_prompt | strict_llm | StrOutputParser()
    flexible_chain = instruction_prompt | flexible_llm | StrOutputParser()
    
    test_format = "1. [Topic] 2. [Definition] 3. [Example]"
    test_content = "Machine learning algorithms that improve through experience"
    
    strict_result = strict_chain.invoke({
        "format": test_format,
        "content": test_content
    })
    
    flexible_result = flexible_chain.invoke({
        "format": test_format, 
        "content": test_content
    })
    
    return {
        "strict_adherence": strict_result,
        "flexible_interpretation": flexible_result
    }

# Run hyperparameter demonstrations
print("🧪 Testing Hyperparameter Effects")
hyperparameter_results = test_hyperparameter_effects()

print("\n🎯 Testing Instruction Adherence")  
adherence_results = test_instruction_adherence()

# Store results in tutorial state
tutorial_state["demo_data"]["hyperparameters"] = hyperparameter_results
tutorial_state["demo_data"]["instruction_adherence"] = adherence_results

print("\n✅ Hyperparameter experimentation complete")
print("📊 Results stored in tutorial_state for analysis")

The examples above demonstrate how hyperparameters create a fundamental tradeoff between instruction following and creative knowledge application. Low temperature models excel at following precise formatting requirements and maintaining consistency across multiple calls, making them ideal for structured data extraction, API responses, and workflows where predictability is paramount. Higher temperature models bring more of the model's training knowledge into play, generating more diverse responses and creative solutions, but at the cost of strict instruction adherence.

This balance becomes critical in agentic systems where you need to decide whether your agent should be a precise executor of specific instructions or a creative problem-solver that can adapt its approach based on context. The choice often depends on your use case: customer service bots might need low-temperature consistency, while creative writing assistants might benefit from higher-temperature diversity.

Now that we understand how to control our model's behavior through prompts and hyperparameters, we need to give our agents the ability to extend beyond their base knowledge and interact with the world. This is where tools come into play - they're what transform a language model from a sophisticated text generator into an active agent that can perform real actions and access current information.

### Tools



Tools are what transform language models from sophisticated text generators into active agents capable of performing real-world actions and accessing live information. Think of tools as the hands and senses of your AI agent - without them, even the most advanced language model is limited to working with only the knowledge it was trained on, which becomes stale the moment training ends. Tools bridge this gap by allowing agents to interact with databases, APIs, web services, file systems, and any other external systems your application needs to work with.

<img src="https://media.licdn.com/dms/image/v2/D4D12AQGyFCaSY8w4Ag/article-cover_image-shrink_720_1280/B4DZYg8dDRHAAI-/0/1744309441965?e=1762992000&v=beta&t=NS3gCnYSTWkxVwnRpHX6tCG7wcXcGgEknNpowIVAo2k" width=700>

The fundamental concept behind tools in agentic systems is function calling (also known as tool calling). Modern language models like GPT-4, Claude, and Gemini have been specifically trained to understand when they need external information or capabilities, and can generate structured function calls with appropriate parameters. When an agent encounters a question about current weather, stock prices, or needs to perform calculations, it doesn't hallucinate an answer - instead, it recognizes the limitation and calls the appropriate tool.

The tool execution process follows a predictable pattern: the agent receives a user request, analyzes what information or actions are needed, determines which tools to use, formats the tool calls with proper parameters, executes the tools, receives the results, and then synthesizes a response using both its knowledge and the tool outputs. This creates a powerful feedback loop where agents can chain multiple tool calls together, use the output of one tool as input to another, and dynamically adapt their approach based on intermediate results.

There are three main categories of tools we'll explore: **built-in tools** that come pre-integrated with language model providers, **explicit tools** that you define and implement yourself, and **Model Context Protocol (MCP) tools** that provide standardized interfaces for complex integrations. Each category serves different purposes and offers varying levels of customization and complexity.

#### Built-in Tools



Built-in tools are native capabilities provided directly by language model providers, eliminating the need for external integrations or custom implementations. Google's Gemini models, for example, come with several powerful built-in tools including Google Search integration, code execution capabilities, and mathematical computation tools. These tools are particularly valuable because they're optimized for the specific model, have minimal latency overhead, and don't require additional API keys or setup beyond your primary model access.

The advantage of built-in tools is their seamless integration - the model provider handles all the complexity of tool execution, result formatting, and error handling. When you enable Google Search for Gemini, the model can perform web searches and incorporate real-time information directly into its responses without any additional code on your part. Similarly, the code execution tool allows Gemini to write and run Python code in a sandboxed environment, making it excellent for data analysis, mathematical calculations, and generating visualizations.

Let's explore how to use Gemini's built-in tools with LangChain:

In [ ]:
# ================================
# BUILT-IN TOOLS DEMONSTRATION  
# ================================
# Showcase Google Gemini's native tool capabilities

def create_builtin_tool_agents():
    """Create agents with different built-in tool configurations"""
    
    # Agent with Google Search integration
    search_agent = ChatGoogleGenerativeAI(
        model="gemini-1.5-pro",
        temperature=0.3,
        google_api_key=os.getenv("GOOGLE_API_KEY"),
        tools=["google_search_retrieval"]
    )
    
    # Agent with code execution capability
    code_agent = ChatGoogleGenerativeAI(
        model="gemini-1.5-pro", 
        temperature=0.1,  # Lower temperature for code reliability
        google_api_key=os.getenv("GOOGLE_API_KEY"),
        tools=["code_execution"]
    )
    
    # Agent with multiple built-in tools
    multi_tool_agent = ChatGoogleGenerativeAI(
        model="gemini-1.5-pro",
        temperature=0.4,
        google_api_key=os.getenv("GOOGLE_API_KEY"),
        tools=["google_search_retrieval", "code_execution"]
    )
    
    return {
        "search_agent": search_agent,
        "code_agent": code_agent, 
        "multi_tool_agent": multi_tool_agent
    }

def test_builtin_tools():
    """Test various built-in tool capabilities"""
    
    agents = create_builtin_tool_agents()
    
    # Test prompts for different tool types
    search_prompt = ChatPromptTemplate.from_messages([
        ("system", "You can search for current information when needed."),
        ("human", "{query}")
    ])
    
    code_prompt = ChatPromptTemplate.from_messages([
        ("system", "You can execute Python code for calculations and analysis."),
        ("human", "{analysis_request}")
    ])
    
    multi_prompt = ChatPromptTemplate.from_messages([
        ("system", "Use search for current info and code execution for calculations."),
        ("human", "{complex_query}")
    ])
    
    # Create chains
    search_chain = search_prompt | agents["search_agent"] | StrOutputParser()
    code_chain = code_prompt | agents["code_agent"] | StrOutputParser()
    multi_chain = multi_prompt | agents["multi_tool_agent"] | StrOutputParser()
    
    # Test queries
    results = {}
    
    try:
        # Search capability test
        search_result = search_chain.invoke({
            "query": "Latest developments in AI safety research 2024"
        })
        results["search_test"] = search_result[:300] + "..."
        
        # Code execution test  
        code_result = code_chain.invoke({
            "analysis_request": """
            Analyze this sales data: [120, 150, 180, 95, 200, 175, 160, 140, 190, 210]
            Calculate mean, median, standard deviation, and identify outliers.
            """
        })
        results["code_test"] = code_result[:300] + "..."
        
        # Multi-tool test
        multi_result = multi_chain.invoke({
            "complex_query": """
            Research current AI market cap data for top 3 companies in 2024,
            then calculate what percentage each represents of total market cap.
            """
        })
        results["multi_tool_test"] = multi_result[:300] + "..."
        
    except Exception as e:
        results["error"] = f"Tool execution error: {str(e)}"
    
    return results

# Execute built-in tools demonstration
print("🔧 Testing Built-in Tool Capabilities")
builtin_results = test_builtin_tools()

for test_name, result in builtin_results.items():
    print(f"\n{test_name.upper()}:")
    print(result)

# Store in tutorial state
tutorial_state["demo_data"]["builtin_tools"] = builtin_results
tutorial_state["current_section"] = "builtin_tools"

print("\n✅ Built-in tools demonstration complete")
print("🏪 Tool results stored in tutorial state")

#### Explicit Tools



While built-in tools provide excellent out-of-the-box functionality, the real power of agentic systems comes from creating custom tools tailored to your specific use case. Explicit tools are functions you define and implement yourself, giving agents the ability to interact with your databases, APIs, business logic, or any other systems your application requires. This is where agents transform from general-purpose assistants into specialized experts for your domain.

The process of creating explicit tools involves defining the tool's interface (what parameters it accepts and what it returns), implementing the actual functionality, and then registering the tool with your agent framework. LangChain makes this process straightforward through its `@tool` decorator and `Tool` class, which handle the integration details while letting you focus on the business logic.

<img src="https://miro.medium.com/v2/resize:fit:2000/1*fu9Lu8D8DLnVFPAWg7N0jQ.png" width=700>

When designing explicit tools, it's important to think about granularity and composability. Rather than creating one massive tool that does everything, it's better to create focused tools that do one thing well and can be combined. For example, instead of a single "manage_database" tool, you might create separate "query_user", "update_inventory", and "calculate_metrics" tools that can work together.

Let's explore how to create and use explicit tools with LangChain and Gemini:

In [ ]:
# ================================
# EXPLICIT TOOLS IMPLEMENTATION
# ================================
# Create custom tools for specific business logic and integrations

def create_custom_tools():
    """Define custom tools using @tool decorator for explicit functionality"""
    
    @tool
    def get_weather(city: str, country: str = "US") -> str:
        """
        Get current weather information for a specified city.
        
        Args:
            city: The name of the city to get weather for
            country: The country code (default: US)
        
        Returns:
            JSON string with weather information
        """
        # Simulate weather API call - replace with real API in production
        weather_conditions = ["sunny", "cloudy", "rainy", "snowy", "partly cloudy"]
        temperature = random.randint(-10, 35)
        condition = random.choice(weather_conditions)
        
        weather_data = {
            "city": city,
            "country": country,
            "temperature": temperature,
            "condition": condition,
            "humidity": random.randint(30, 90),
            "timestamp": datetime.datetime.now().isoformat()
        }
        
        return json.dumps(weather_data, indent=2)

    @tool
    def calculate_compound_interest(principal: float, rate: float, time: int, compounds_per_year: int = 1) -> str:
        """
        Calculate compound interest using the formula: A = P(1 + r/n)^(nt)
        
        Mathematical Foundation:
        A = P(1 + r/n)^(nt)
        Where:
        - A = final amount
        - P = principal (initial investment) 
        - r = annual interest rate (as decimal)
        - n = number of times interest compounds per year
        - t = time in years
        
        Args:
            principal: Initial investment amount
            rate: Annual interest rate (as decimal, e.g., 0.05 for 5%)
            time: Number of years
            compounds_per_year: Compounding frequency (default: 1)
        
        Returns:
            Formatted string with calculation details
        """
        # Apply compound interest formula
        amount = principal * (1 + rate/compounds_per_year) ** (compounds_per_year * time)
        interest_earned = amount - principal
        
        result = {
            "principal": principal,
            "annual_rate": f"{rate*100}%", 
            "time_years": time,
            "compounds_per_year": compounds_per_year,
            "final_amount": round(amount, 2),
            "interest_earned": round(interest_earned, 2),
            "total_return_percentage": round((interest_earned/principal)*100, 2)
        }
        
        return json.dumps(result, indent=2)

    @tool  
    def search_user_database(query: str, user_type: str = "all") -> str:
        """
        Search a simulated user database for customer information.
        
        Args:
            query: Search term (name, email, or ID)
            user_type: Filter by user type - "premium", "basic", or "all"
        
        Returns:
            JSON string with user information
        """
        # Mock database - replace with actual database queries in production
        mock_users = [
            {"id": "001", "name": "Alice Johnson", "email": "alice@email.com", "type": "premium", "status": "active"},
            {"id": "002", "name": "Bob Smith", "email": "bob@email.com", "type": "basic", "status": "active"}, 
            {"id": "003", "name": "Carol Davis", "email": "carol@email.com", "type": "premium", "status": "inactive"},
            {"id": "004", "name": "David Wilson", "email": "david@email.com", "type": "basic", "status": "active"}
        ]
        
        # Apply user type filter
        if user_type != "all":
            mock_users = [user for user in mock_users if user["type"] == user_type]
        
        # Search logic with fuzzy matching
        results = []
        query_lower = query.lower()
        for user in mock_users:
            if (query_lower in user["name"].lower() or 
                query_lower in user["email"].lower() or 
                query_lower == user["id"]):
                results.append(user)
        
        return json.dumps({"query": query, "results": results}, indent=2)
    
    return [get_weather, calculate_compound_interest, search_user_database]

def create_tool_agent(tools_list):
    """Create an agent executor with custom tools"""
    
    tool_prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a helpful assistant with access to several specialized tools:
        
        🌤️  get_weather: Get current weather for any city
        💰 calculate_compound_interest: Calculate investment returns with compound interest
        👥 search_user_database: Look up customer information in database
        
        Use these tools when needed to provide accurate, helpful responses.
        Always explain which tool you're using and why.
        Format JSON data nicely for users."""),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ])
    
    agent = create_tool_calling_agent(llm, tools_list, tool_prompt)
    
    agent_executor = AgentExecutor(
        agent=agent, 
        tools=tools_list, 
        verbose=True,
        handle_parsing_errors=True
    )
    
    return agent_executor

def test_explicit_tools():
    """Test the custom tools with various scenarios"""
    
    custom_tools = create_custom_tools()
    tool_agent = create_tool_agent(custom_tools)
    
    test_scenarios = [
        {
            "name": "Weather Query",
            "input": "What's the weather like in Tokyo, Japan right now?"
        },
        {
            "name": "Financial Calculation", 
            "input": "If I invest $10,000 at 6% annual interest compounded monthly for 10 years, what will I have?"
        },
        {
            "name": "Database Search",
            "input": "Can you find information about user Alice in our database?"
        },
        {
            "name": "Multi-Tool Chain",
            "input": """I need help with:
            1. Weather in San Francisco
            2. Find premium users named David 
            3. Calculate $5000 invested at 4.5% annually for 5 years"""
        }
    ]
    
    results = {}
    
    for scenario in test_scenarios:
        print(f"\n🧪 Testing: {scenario['name']}")
        try:
            response = tool_agent.invoke({"input": scenario["input"]})
            results[scenario["name"]] = response["output"]
            print(f"✅ Success: {response['output'][:150]}...")
        except Exception as e:
            results[scenario["name"]] = f"Error: {str(e)}"
            print(f"❌ Error: {str(e)}")
    
    return results, custom_tools

# Execute explicit tools demonstration
print("🛠️  Creating Custom Tools")
explicit_results, custom_tools = test_explicit_tools()

# Store in tutorial state
tutorial_state["demo_data"]["explicit_tools"] = explicit_results
tutorial_state["tools"] = {"custom_tools": custom_tools}
tutorial_state["current_section"] = "explicit_tools"

print("\n✅ Explicit tools implementation complete")
print("🎯 Custom tools integrated and tested successfully")

#### Model Context Protocol (MCP)



Model Context Protocol (MCP) represents the next evolution in AI tool integration, providing a standardized way for AI applications to securely connect to data sources and tools. Think of MCP as a universal translator that allows any AI system to communicate with any external service through a common protocol, eliminating the need for custom integrations for each tool or data source.

<img src="https://pbs.twimg.com/tweet_video_thumb/Gl7C44tXYAAdDSJ.jpg" width=700>

MCP was developed by Anthropic to solve the fragmentation problem in AI tool ecosystems. Before MCP, every AI application had to implement its own custom integrations for databases, APIs, file systems, and other external resources. This led to duplicated effort, security inconsistencies, and tools that only worked with specific AI platforms. MCP standardizes these interactions through a client-server architecture where MCP servers expose resources (like databases or file systems) and tools (like calculators or API clients) through a uniform interface.

The protocol operates on JSON-RPC 2.0, enabling real-time, bidirectional communication between AI applications (MCP clients) and external resources (MCP servers). This means your agent can not only call tools but also receive real-time updates, notifications, and streaming data from external systems. The security model is built around explicit capability declarations and sandboxed execution, ensuring that agents can only access resources they've been explicitly granted permission to use.

What makes MCP particularly powerful for RAG and agentic systems is its ability to provide **contextual data access**. Instead of just calling functions, MCP servers can expose rich contextual information about resources - like database schemas, file structures, or API capabilities - allowing agents to make more informed decisions about how to interact with external systems.

Let's explore how to integrate MCP servers with LangChain and Gemini. For this example, we'll use the MCP SDK to create a simple server and then connect to it:

In [ ]:
# Note: This example demonstrates MCP concepts. In practice, you would install:
# pip install mcp langchain-mcp

# For now, we'll simulate MCP functionality to understand the concepts
from typing import Any, Dict, List
import json
import asyncio
from dataclasses import dataclass

# Simulate an MCP server interface
@dataclass
class MCPResource:
    """Represents a resource exposed by an MCP server"""
    uri: str
    name: str
    description: str
    mime_type: str

@dataclass 
class MCPTool:
    """Represents a tool exposed by an MCP server"""
    name: str
    description: str
    input_schema: Dict[str, Any]

class MockMCPServer:
    """Simulated MCP server for demonstration purposes"""
    
    def __init__(self, name: str):
        self.name = name
        self.resources: List[MCPResource] = []
        self.tools: List[MCPTool] = []
        
    def add_resource(self, resource: MCPResource):
        self.resources.append(resource)
        
    def add_tool(self, tool: MCPTool):
        self.tools.append(tool)
        
    def list_resources(self) -> List[Dict[str, Any]]:
        """List all available resources"""
        return [
            {
                "uri": r.uri,
                "name": r.name, 
                "description": r.description,
                "mimeType": r.mime_type
            } for r in self.resources
        ]
        
    def list_tools(self) -> List[Dict[str, Any]]:
        """List all available tools"""
        return [
            {
                "name": t.name,
                "description": t.description,
                "inputSchema": t.input_schema
            } for t in self.tools
        ]
        
    def read_resource(self, uri: str) -> str:
        """Read content from a resource"""
        # Simulate resource reading
        if "customer_db" in uri:
            return json.dumps({
                "customers": [
                    {"id": 1, "name": "John Doe", "email": "john@example.com", "tier": "gold"},
                    {"id": 2, "name": "Jane Smith", "email": "jane@example.com", "tier": "silver"}
                ],
                "schema": {
                    "id": "integer",
                    "name": "string", 
                    "email": "string",
                    "tier": "string"
                }
            })
        elif "inventory" in uri:
            return json.dumps({
                "items": [
                    {"sku": "A001", "name": "Laptop", "quantity": 50, "price": 999.99},
                    {"sku": "A002", "name": "Mouse", "quantity": 200, "price": 29.99}
                ]
            })
        return "Resource not found"
        
    def call_tool(self, tool_name: str, arguments: Dict[str, Any]) -> str:
        """Execute a tool with given arguments"""
        if tool_name == "query_analytics":
            metric = arguments.get("metric", "sales")
            period = arguments.get("period", "month")
            return json.dumps({
                "metric": metric,
                "period": period,
                "value": 150000 if metric == "sales" else 1200,
                "trend": "increasing",
                "timestamp": "2024-10-22T10:00:00Z"
            })
        elif tool_name == "send_notification":
            return json.dumps({
                "status": "sent",
                "recipient": arguments.get("recipient"),
                "message": arguments.get("message"),
                "delivery_id": "notify_12345"
            })
        return json.dumps({"error": "Tool not found"})

# Create a mock MCP server with business resources and tools
business_mcp = MockMCPServer("business_system")

# Add resources (data sources the agent can read)
business_mcp.add_resource(MCPResource(
    uri="mcp://business/customer_db",
    name="Customer Database",
    description="Customer information and account details", 
    mime_type="application/json"
))

business_mcp.add_resource(MCPResource(
    uri="mcp://business/inventory",
    name="Inventory System", 
    description="Product inventory and stock levels",
    mime_type="application/json"
))

# Add tools (actions the agent can perform)
business_mcp.add_tool(MCPTool(
    name="query_analytics",
    description="Query business analytics and metrics",
    input_schema={
        "type": "object",
        "properties": {
            "metric": {"type": "string", "enum": ["sales", "users", "revenue"]},
            "period": {"type": "string", "enum": ["day", "week", "month", "year"]}
        },
        "required": ["metric"]
    }
))

business_mcp.add_tool(MCPTool(
    name="send_notification", 
    description="Send notifications to users or systems",
    input_schema={
        "type": "object",
        "properties": {
            "recipient": {"type": "string"},
            "message": {"type": "string"},
            "priority": {"type": "string", "enum": ["low", "medium", "high"]}
        },
        "required": ["recipient", "message"]
    }
))

print("=== MCP Server Created ===")
print(f"Server: {business_mcp.name}")
print(f"Resources: {len(business_mcp.resources)}")
print(f"Tools: {len(business_mcp.tools)}")

# List available resources and tools
print("\n=== Available Resources ===")
for resource in business_mcp.list_resources():
    print(f"- {resource['name']}: {resource['description']}")
    
print("\n=== Available Tools ===") 
for tool in business_mcp.list_tools():
    print(f"- {tool['name']}: {tool['description']}")

In [ ]:
# Create LangChain tools that interface with our MCP server
# This demonstrates how MCP servers can be integrated into LangChain workflows

@tool
def mcp_read_resource(resource_name: str) -> str:
    """
    Read data from MCP server resources like databases or file systems.
    
    Args:
        resource_name: Name of the resource to read (customer_db, inventory)
    
    Returns:
        JSON string with resource data
    """
    uri_map = {
        "customer_db": "mcp://business/customer_db",
        "customers": "mcp://business/customer_db", 
        "inventory": "mcp://business/inventory",
        "products": "mcp://business/inventory"
    }
    
    uri = uri_map.get(resource_name.lower())
    if not uri:
        return json.dumps({"error": f"Resource '{resource_name}' not found"})
        
    return business_mcp.read_resource(uri)

@tool
def mcp_query_analytics(metric: str, period: str = "month") -> str:
    """
    Query business analytics through MCP server.
    
    Args:
        metric: The metric to query (sales, users, revenue)
        period: Time period for the metric (day, week, month, year)
    
    Returns:
        JSON string with analytics data
    """
    return business_mcp.call_tool("query_analytics", {
        "metric": metric,
        "period": period
    })

@tool  
def mcp_send_notification(recipient: str, message: str, priority: str = "medium") -> str:
    """
    Send notifications through MCP server.
    
    Args:
        recipient: Who to send the notification to
        message: The notification message
        priority: Priority level (low, medium, high)
    
    Returns:
        JSON string with delivery confirmation
    """
    return business_mcp.call_tool("send_notification", {
        "recipient": recipient,
        "message": message,
        "priority": priority
    })

# Create MCP-enabled tools list
mcp_tools = [mcp_read_resource, mcp_query_analytics, mcp_send_notification]

# Create an agent that can use MCP tools
mcp_llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0.2,
    google_api_key=os.getenv("GOOGLE_API_KEY")
)

mcp_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a business intelligence assistant with access to company systems through MCP.
    
    Available MCP resources:
    - customer_db: Customer information and account details
    - inventory: Product inventory and stock levels
    
    Available MCP tools:
    - mcp_query_analytics: Get business metrics and analytics
    - mcp_send_notification: Send notifications to users or systems
    - mcp_read_resource: Read data from company databases and systems
    
    Use these tools to provide comprehensive business insights and take actions when requested.
    Always format data nicely and explain what you're doing."""),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

mcp_agent = create_tool_calling_agent(mcp_llm, mcp_tools, mcp_prompt)
mcp_executor = AgentExecutor(
    agent=mcp_agent,
    tools=mcp_tools,
    verbose=True,
    handle_parsing_errors=True
)

print("=== MCP-Enabled Agent Created ===")
print("Agent ready with MCP server integration")

In [ ]:
# Test the MCP-enabled agent with business scenarios

print("=== Test 1: Customer Data Analysis ===")
customer_analysis = mcp_executor.invoke({
    "input": "Can you analyze our customer data? I want to see the customer information and understand our customer tiers."
})
print("Response:", customer_analysis['output'])

print("\n=== Test 2: Business Analytics ===")
analytics_query = mcp_executor.invoke({
    "input": "What were our sales metrics for this month? Also check user metrics."
})
print("Response:", analytics_query['output'])

print("\n=== Test 3: Inventory Management ===") 
inventory_check = mcp_executor.invoke({
    "input": "Check our current inventory levels and identify any products that might need restocking."
})
print("Response:", inventory_check['output'])

print("\n=== Test 4: Complex Business Workflow ===")
complex_workflow = mcp_executor.invoke({
    "input": """I need a comprehensive business report:
    1. Check our customer database for gold tier customers
    2. Get our current sales metrics
    3. Review inventory levels
    4. If sales are good and we have low inventory, send a notification to 'inventory-team@company.com' about restocking
    
    Please provide a summary with actionable insights."""
})
print("Response:", complex_workflow['output'])

The examples above demonstrate the power of tools in transforming language models into capable agents. We've seen how **built-in tools** provide immediate capabilities with minimal setup, **explicit tools** offer complete customization for your specific needs, and **MCP tools** enable standardized integration with complex systems while maintaining security and scalability.

The key insight is that tools are what bridge the gap between language model intelligence and real-world utility. Without tools, even the most sophisticated language model is limited to generating text based on its training data. With tools, agents become active participants in your business processes, capable of querying databases, performing calculations, calling APIs, and taking actions in response to user needs.

As we design agentic systems, the choice between different tool types depends on your specific requirements:
- Use **built-in tools** when the model provider offers functionality that meets your needs
- Create **explicit tools** when you need custom integration with your specific systems  
- Implement **MCP tools** when you need standardized, scalable integrations across multiple AI applications

Now that our agents can take actions in the world through tools, we need to ensure they can maintain context and remember information across interactions. This is where memory and context management become crucial for building agents that can handle complex, multi-step workflows and maintain coherent conversations over time.

### Context Management



Context management is the cognitive backbone of sophisticated agents, determining how they maintain awareness of ongoing conversations, remember past interactions, and build upon previous knowledge to provide coherent, contextually relevant responses. Without proper context management, even the most capable agents become like individuals with severe short-term memory loss—they might excel at individual tasks but fail to maintain meaningful, coherent interactions over time.

Think of context management as the difference between having a conversation with a knowledgeable expert who remembers your entire discussion versus repeatedly starting fresh with someone who has no recollection of what you've already covered. The former builds understanding progressively, references earlier points, and adapts their communication based on your evolving needs. The latter, while potentially knowledgeable, forces you to repeat yourself and cannot build on the conversational foundation you've established.

In agentic systems, context management becomes even more critical because agents need to coordinate information across multiple tool calls, maintain state during complex workflows, and remember important details that influence future decisions. An agent helping with financial planning needs to remember your risk tolerance, investment timeline, and previous decisions to provide consistent advice. A customer service agent should recall your account history, previous issues, and preferences to deliver personalized support.

The challenge lies in balancing several competing factors: **memory capacity** (how much information can be retained), **relevance** (what information is most important to keep), **efficiency** (managing token limits and processing costs), and **persistence** (maintaining memory across sessions). Different memory strategies excel in different scenarios, and the best approach often involves combining multiple memory types to create a comprehensive context management system.

<img src="https://substackcdn.com/image/fetch/$s_!AyLS!,f_auto,q_auto:good,fl_progressive:steep/https%3A%2F%2Fsubstack-post-media.s3.amazonaws.com%2Fpublic%2Fimages%2Fb0e3c002-0841-4d5f-9171-3eb63c321824_1600x1224.png" width=700>

Memory systems in agentic applications serve different purposes and have distinct strengths and limitations. Understanding these differences is crucial for selecting the right memory strategy for your specific use case. Let's explore the major categories of memory available in LangChain and how they can be effectively utilized.

**Buffer-based memories** store raw conversation history up to certain limits, providing complete fidelity but consuming significant token space. **Summary-based memories** compress conversation history into concise summaries, trading some detail for efficiency. **Window-based memories** maintain only recent interactions, ensuring relevance while discarding older context. **Token-aware memories** dynamically manage content based on token consumption, balancing completeness with cost constraints.

Each memory type excels in specific scenarios: use buffer memory for short conversations where every detail matters, summary memory for long-running sessions where themes and key decisions need tracking, window memory for task-oriented interactions where only recent context is relevant, and token buffer memory for cost-sensitive applications with unpredictable conversation lengths.

Let's implement and compare these different memory systems:

In [ ]:
# ================================
# MEMORY SYSTEMS COMPREHENSIVE DEMO
# ================================
# Demonstrate different memory strategies and their mathematical foundations

class MemorySystemsDemo:
    """Comprehensive demonstration of different memory strategies"""
    
    def __init__(self, llm):
        self.llm = llm
        self.memory_systems = {}
        self.test_results = {}
    
    def create_all_memory_types(self):
        """Initialize all memory system types"""
        
        # 1. ConversationBufferMemory - Complete history storage
        # Memory complexity: O(n) where n = total messages
        buffer_memory = ConversationBufferMemory(
            memory_key="chat_history",
            return_messages=True
        )
        
        # 2. ConversationSummaryMemory - Compressed summaries
        # Memory complexity: O(log n) with periodic compression
        summary_memory = ConversationSummaryMemory(
            llm=self.llm,
            memory_key="chat_history", 
            return_messages=True
        )
        
        # 3. ConversationBufferWindowMemory - Sliding window 
        # Memory complexity: O(k) where k = window size
        window_memory = ConversationBufferWindowMemory(
            k=3,  # Keep last 3 conversation pairs
            memory_key="chat_history",
            return_messages=True
        )
        
        # 4. ConversationTokenBufferMemory - Token-aware pruning
        # Memory complexity: O(m) where m = max_token_limit
        token_memory = ConversationTokenBufferMemory(
            llm=self.llm,
            max_token_limit=500,
            memory_key="chat_history",
            return_messages=True
        )
        
        # 5. ConversationEntityMemory - Entity relationship tracking
        # Memory complexity: O(e) where e = number of entities
        entity_store = InMemoryEntityStore()
        entity_memory = ConversationEntityMemory(
            llm=self.llm,
            entity_store=entity_store,
            memory_key="chat_history",
            return_messages=True
        )
        
        self.memory_systems = {
            "buffer": buffer_memory,
            "summary": summary_memory,
            "window": window_memory,
            "token": token_memory,
            "entity": entity_memory
        }
        
        return self.memory_systems
    
    def create_memory_conversations(self):
        """Create conversation chains for each memory type"""
        
        conversations = {}
        
        for memory_name, memory_system in self.memory_systems.items():
            conversation = ConversationChain(
                llm=self.llm,
                memory=memory_system,
                verbose=False  # Reduced verbosity for cleaner output
            )
            conversations[memory_name] = conversation
        
        return conversations
    
    def test_memory_scenarios(self):
        """Test different memory systems with various conversation patterns"""
        
        conversations = self.create_memory_conversations()
        
        # Test scenarios for different memory characteristics
        scenarios = {
            "short_detailed": [
                "I'm planning a wedding for next summer.",
                "The venue is in California, budget is $50,000.",
                "We expect 150 guests, mostly family and close friends."
            ],
            "long_business": [
                "Let's discuss the Q4 marketing strategy for TechCorp.",
                "Our target is to increase market share by 15% this quarter.", 
                "The main competitors are DataSys and CloudFlow Solutions.",
                "We have a budget of $2M for digital advertising campaigns.",
                "Sarah Johnson is the marketing director, she prefers social media focus.",
                "The product launch is scheduled for December 15th.",
                "We need to coordinate with the sales team led by Mike Chen.",
                "Previous campaigns showed 12% conversion rates on LinkedIn ads."
            ],
            "entity_heavy": [
                "I work with Microsoft on cloud projects. The PM is Alice Wang.",
                "Alice mentioned they're migrating to Azure Service Bus.",
                "The project budget is $500K and timeline is 6 months.",
                "We also collaborate with Amazon Web Services team.",
                "John Smith from AWS handles the integration requirements.",
                "Microsoft wants completion by March 2025 for their fiscal year."
            ]
        }
        
        results = {}
        
        for scenario_name, messages in scenarios.items():
            print(f"\n🧪 Testing Scenario: {scenario_name}")
            scenario_results = {}
            
            for memory_type, conversation in conversations.items():
                print(f"  Testing {memory_type} memory...")
                
                # Reset conversation for each test
                conversation.memory.clear()
                
                responses = []
                for message in messages:
                    response = conversation.predict(input=message)
                    responses.append(response[:100] + "..." if len(response) > 100 else response)
                
                # Analyze memory state
                memory_state = self.analyze_memory_state(conversation.memory, memory_type)
                
                scenario_results[memory_type] = {
                    "responses": responses,
                    "memory_analysis": memory_state
                }
            
            results[scenario_name] = scenario_results
        
        self.test_results = results
        return results
    
    def analyze_memory_state(self, memory, memory_type):
        """Analyze the current state of a memory system"""
        
        analysis = {"type": memory_type}
        
        try:
            if hasattr(memory, 'chat_memory'):
                analysis["message_count"] = len(memory.chat_memory.messages)
                
            if memory_type == "buffer":
                # Buffer memory: complete message history
                analysis["storage_type"] = "complete_history"
                analysis["growth_pattern"] = "linear"
                
            elif memory_type == "summary":
                # Summary memory: compressed representation
                analysis["storage_type"] = "compressed_summary"
                analysis["growth_pattern"] = "logarithmic"
                if hasattr(memory, 'buffer'):
                    analysis["current_summary"] = memory.buffer[:100] + "..." if memory.buffer else "No summary yet"
                    
            elif memory_type == "window": 
                # Window memory: fixed-size sliding window
                analysis["storage_type"] = "sliding_window"
                analysis["growth_pattern"] = "constant"
                analysis["window_size"] = memory.k * 2  # k conversation pairs
                
            elif memory_type == "token":
                # Token memory: token-aware pruning
                analysis["storage_type"] = "token_limited"
                analysis["growth_pattern"] = "bounded"
                analysis["max_tokens"] = memory.max_token_limit
                
            elif memory_type == "entity":
                # Entity memory: relationship tracking
                analysis["storage_type"] = "entity_graph"
                analysis["growth_pattern"] = "entity_proportional"
                if hasattr(memory, 'entity_store'):
                    entities = list(memory.entity_store.store.keys())
                    analysis["tracked_entities"] = entities[:5]  # Show first 5
                    analysis["entity_count"] = len(entities)
                    
        except Exception as e:
            analysis["error"] = str(e)
            
        return analysis

# Initialize and run memory systems demonstration
print("🧠 Initializing Memory Systems Demo")
memory_demo = MemorySystemsDemo(llm)
memory_systems = memory_demo.create_all_memory_types()

print("🔄 Testing Memory Systems with Different Scenarios")
memory_test_results = memory_demo.test_memory_scenarios()

# Store comprehensive results
tutorial_state["memory_systems"] = memory_systems
tutorial_state["demo_data"]["memory_tests"] = memory_test_results
tutorial_state["current_section"] = "memory_systems"

print("\n✅ Memory Systems Comprehensive Demo Complete")
print("🎯 All memory types tested and analyzed")
print("📊 Results include complexity analysis and performance characteristics")

##### ConversationBufferMemory


ConversationBufferMemory is the most straightforward memory implementation, storing the complete conversation history without any compression or filtering. This memory type maintains perfect fidelity to the original conversation, preserving every nuance, detail, and context from the interaction history. It's like having a perfect recording of every word spoken in a meeting—nothing is lost, but the storage requirements can become substantial.

The primary advantage of buffer memory is its completeness and simplicity. Every message, response, and interaction detail remains available for the agent to reference, making it ideal for scenarios where precise recall is critical—think legal consultations, medical histories, or technical support where missing details could have significant consequences. The agent can refer back to exact phrasings, specific numbers, or detailed explanations provided earlier in the conversation.

However, buffer memory's strength becomes its weakness in extended conversations. As the conversation grows, token consumption increases linearly, potentially exceeding model context limits and significantly increasing costs. For models with 4K token limits, a detailed conversation might fill the entire context window, leaving little space for actual reasoning and response generation.

##### ConversationSummaryMemory



ConversationSummaryMemory addresses the scalability limitations of buffer memory by maintaining a running summary of the conversation rather than storing every individual message. Think of it as having an intelligent note-taker who captures the key themes, decisions, and important details while filtering out redundant or less relevant information. This approach allows for indefinitely long conversations while maintaining reasonable token consumption.

The summary mechanism works by periodically condensing older conversation history into concise summaries using the language model itself. When the conversation reaches a certain length, the memory system takes the oldest messages, generates a summary of their content, and replaces the original messages with this compressed representation. New messages continue to be stored in full until the next summarization cycle.

This approach excels in scenarios where conversation themes and key decisions matter more than exact wording—think project planning sessions, brainstorming meetings, or ongoing consulting relationships where the agent needs to remember overall context and previous decisions but doesn't need verbatim recall of every exchange.

##### ConversationBufferWindowMemory

ConversationBufferWindowMemory takes a different approach to managing conversation length by maintaining only the most recent N interactions in full detail while discarding older messages entirely. This sliding window approach ensures consistent performance and predictable token usage, making it ideal for applications where recent context is most relevant and older interactions can be safely forgotten.

The window size (k parameter) determines how many of the most recent message pairs to retain. For example, with k=3, the memory stores the last 3 human messages and their corresponding AI responses, totaling 6 messages. When a new interaction occurs, the oldest message pair is dropped to make room for the new one, maintaining a constant memory footprint.

This memory type excels in task-oriented conversations where the immediate context matters most—think customer service interactions, troubleshooting sessions, or iterative design processes where each step builds on the previous few but doesn't require the entire conversation history.

In [ ]:
# ConversationBufferWindowMemory - maintains only recent interactions
window_memory = ConversationBufferWindowMemory(
    k=3,  # Keep last 3 conversation pairs (6 messages total)
    memory_key="chat_history",
    return_messages=True
)

window_conversation = ConversationChain(
    llm=memory_llm,
    memory=window_memory,
    verbose=True
)

print("=== ConversationBufferWindowMemory Demo ===")

# Simulate a troubleshooting conversation
troubleshooting_steps = [
    "My laptop won't start. The screen stays black when I press the power button.",
    "I can see a small LED light on the laptop, but nothing else happens.",
    "I tried holding the power button for 30 seconds, but no change.",
    "Let me try removing the battery and plugging in just the power adapter.",
    "That worked! The laptop started. So it seems like a battery issue?",
    "Should I get a replacement battery or could this be something else?",
    "How can I test if the battery is completely dead or just needs charging?"
]

for i, user_input in enumerate(troubleshooting_steps, 1):
    response = window_conversation.predict(input=user_input)
    print(f"Step {i}: Completed")
    
    # Show memory contents after each step
    memory_contents = window_memory.chat_memory.messages
    print(f"Memory contains {len(memory_contents)} messages (window size = {window_memory.k * 2})")

print(f"\n=== Final Memory State ===")
print("Window memory final contents:")
for i, message in enumerate(window_memory.chat_memory.messages):
    print(f"Message {i+1}: {type(message).__name__} - {message.content[:80]}...")

##### ConversationTokenBufferMemory



ConversationTokenBufferMemory provides the most sophisticated approach to memory management by dynamically adjusting conversation history based on token consumption rather than fixed message counts or arbitrary summarization triggers. This memory type continuously monitors token usage and intelligently removes older messages when approaching the specified token limit, ensuring optimal utilization of the model's context window while maintaining as much relevant history as possible.

The key innovation here is token-aware pruning—the memory system counts tokens in the conversation history and removes the oldest messages when the total approaches the configured limit. This ensures you never exceed context limits while maximizing the amount of conversation history available to the model. It's like having an intelligent editor who knows exactly how much content fits in the available space and makes informed decisions about what to keep.

This approach is particularly valuable in production applications where token costs matter and conversation lengths vary unpredictably. It provides the reliability of never exceeding context limits with the efficiency of using available context space optimally.

In [ ]:
# ConversationTokenBufferMemory - manages memory based on token count
token_memory = ConversationTokenBufferMemory(
    llm=memory_llm,
    max_token_limit=500,  # Keep conversation under 500 tokens
    memory_key="chat_history",
    return_messages=True
)

token_conversation = ConversationChain(
    llm=memory_llm,
    memory=token_memory,
    verbose=True
)

print("=== ConversationTokenBufferMemory Demo ===")

# Test with progressively longer inputs to see token management
test_inputs = [
    "What is artificial intelligence?",
    "Can you explain machine learning in detail, including supervised and unsupervised learning approaches?",
    "I'm interested in deep learning. Can you walk me through neural networks, backpropagation, and how gradient descent works?",
    "What about transformers and attention mechanisms? How do they work in modern language models?",
    "Can you compare different AI architectures like CNNs, RNNs, LSTMs, and transformers in terms of their strengths and use cases?"
]

for i, user_input in enumerate(test_inputs, 1):
    print(f"\n--- Interaction {i} ---")
    print(f"Input length: ~{len(user_input.split()) * 1.3:.0f} tokens (estimated)")
    
    response = token_conversation.predict(input=user_input)
    
    # Check token usage
    current_messages = token_memory.chat_memory.messages
    estimated_tokens = sum(len(msg.content.split()) * 1.3 for msg in current_messages)
    
    print(f"Memory contains {len(current_messages)} messages")
    print(f"Estimated tokens in memory: {estimated_tokens:.0f} / {token_memory.max_token_limit}")
    print(f"Response preview: {response[:100]}...")

print("\n=== Token Management Analysis ===")
print("Token buffer memory automatically pruned older messages to stay within limits")

##### ConversationEntityMemory



ConversationEntityMemory represents a more sophisticated approach to context management by focusing on entities—specific people, places, organizations, concepts, or objects—mentioned throughout the conversation. Rather than treating all information equally, this memory system identifies and tracks important entities, maintaining detailed information about each one and their relationships to the ongoing conversation.

Think of entity memory as having a smart assistant who keeps detailed notes about every person, company, project, or concept you discuss, building a rich knowledge graph of your conversation topics. When you mention "the Johnson project" or "Sarah from marketing," the system retrieves all relevant context about these entities from previous discussions, even if they were mentioned weeks ago.

This approach excels in complex, ongoing relationships where tracking multiple entities and their evolving attributes is crucial—think sales conversations tracking multiple clients and deals, project management discussions involving various stakeholders and deliverables, or research conversations where you're building understanding about multiple related concepts over time.

In [ ]:
# ConversationEntityMemory - tracks entities and their relationships
entity_store = InMemoryEntityStore()
entity_memory = ConversationEntityMemory(
    llm=memory_llm,
    entity_store=entity_store,
    memory_key="chat_history",
    return_messages=True
)

entity_conversation = ConversationChain(
    llm=memory_llm,
    memory=entity_memory,
    verbose=True
)

print("=== ConversationEntityMemory Demo ===")

# Simulate a business conversation with multiple entities
business_conversations = [
    "I'm working with TechCorp on a new software project. The CEO is Maria Rodriguez.",
    "Maria mentioned they need the mobile app completed by December 15th for their Q4 launch.",
    "The project budget is $250,000 and we have a team of 5 developers assigned.",
    "We're also collaborating with DataSys Inc for the backend infrastructure. Their CTO is James Chen.",
    "James told me they use AWS for hosting and PostgreSQL for the database.",
    "TechCorp wants to integrate with their existing CRM system that was built by Solutions Ltd.",
    "Maria is concerned about security compliance since they handle financial data.",
    "The mobile app will have both iOS and Android versions, targeting business users."
]

for i, user_input in enumerate(business_conversations, 1):
    response = entity_conversation.predict(input=user_input)
    print(f"Business discussion {i} completed")

# Examine the entities that were tracked
print("\n=== Entity Memory Analysis ===")
print("Entities identified and tracked:")
entity_data = entity_memory.entity_store.store
for entity_name, entity_info in entity_data.items():
    print(f"- {entity_name}: {entity_info}")

# Test entity recall with a follow-up question
print("\n=== Entity Recall Test ===")
followup_response = entity_conversation.predict(
    input="What was Maria's deadline for the project and what's their budget?"
)
print("Follow-up response:", followup_response[:200] + "...")

##### CombinedMemory and Advanced Patterns



Real-world applications often benefit from combining multiple memory strategies to create sophisticated context management systems that leverage the strengths of different approaches while mitigating their individual limitations. CombinedMemory allows you to orchestrate multiple memory systems simultaneously, creating layered context awareness that can handle both immediate needs and long-term relationship building.

For example, you might combine ConversationBufferWindowMemory for immediate context with ConversationEntityMemory for long-term entity tracking, plus a custom memory component for domain-specific information. This creates a multi-layered memory architecture where recent interactions provide immediate context, entity memory maintains relationship continuity, and specialized memory components handle domain-specific requirements like user preferences or system configurations.

Let's implement a combined memory system that demonstrates this architectural approach:

In [ ]:
# CombinedMemory - orchestrate multiple memory systems
from langchain.memory import SimpleMemory

# Create individual memory components
recent_memory = ConversationBufferWindowMemory(
    k=2, memory_key="recent_history", return_messages=True
)

entity_tracker = ConversationEntityMemory(
    llm=memory_llm,
    entity_store=InMemoryEntityStore(),
    memory_key="entities",
    return_messages=False  # Just track entities, don't return full chat
)

# Create a simple memory for user preferences
preferences_memory = SimpleMemory(
    memories={"user_preferences": "No specific preferences set yet"}
)

# Combine all memory systems
combined_memory = CombinedMemory(
    memories=[recent_memory, entity_tracker, preferences_memory]
)

# Custom prompt template that utilizes all memory types
combined_prompt = PromptTemplate(
    input_variables=["recent_history", "entities", "user_preferences", "input"],
    template="""You are an AI assistant with comprehensive memory capabilities.

Recent Conversation: {recent_history}

Known Entities: {entities}

User Preferences: {user_preferences}

Based on this context, respond to: {input}

Be conversational and reference relevant context from memory when appropriate."""
)

# Create chain with combined memory
combined_chain = ConversationChain(
    llm=memory_llm,
    memory=combined_memory,
    prompt=combined_prompt,
    verbose=True
)

print("=== CombinedMemory Demo ===")

# Test the combined memory system
test_conversation = [
    "Hi, I'm Sarah and I prefer concise responses. I'm working on a Python project.",
    "I need help with data analysis using pandas. Can you recommend some techniques?",
    "Actually, I'm working with customer data for my company TechFlow Solutions.",
    "Our CEO Mike Johnson wants insights on customer retention patterns.",
    "Can you suggest a visualization approach for this data?"
]

for i, user_input in enumerate(test_conversation, 1):
    response = combined_chain.predict(input=user_input)
    print(f"Combined memory interaction {i} completed")

print("\n=== Memory System Analysis ===")
print("Combined memory successfully integrated:")
print("- Recent conversation context")
print("- Entity tracking across sessions") 
print("- User preference management")
print("- Seamless coordination between memory types")

The examples above demonstrate the spectrum of memory management strategies available for agentic systems. Each approach serves different purposes and excels in specific scenarios:

**ConversationBufferMemory** provides perfect recall for short conversations where every detail matters, but becomes expensive in extended interactions. **ConversationSummaryMemory** enables indefinitely long conversations by maintaining key themes while sacrificing some detail. **ConversationBufferWindowMemory** offers predictable performance by keeping only recent context, ideal for task-oriented interactions. **ConversationTokenBufferMemory** provides optimal context utilization with cost control, perfect for production applications.

**ConversationEntityMemory** excels at tracking relationships and building long-term understanding, while **CombinedMemory** allows sophisticated orchestration of multiple memory strategies. The choice depends on your specific requirements: conversation length, cost constraints, detail requirements, and the importance of long-term relationship building.

In practice, most production agentic systems benefit from combining multiple memory approaches, using recent memory for immediate context, entity memory for relationship continuity, and token-aware management for cost control. This creates robust context management that adapts to different conversation patterns while maintaining performance and reliability.

Now that our agents have sophisticated memory capabilities, let's explore how they can develop and refine specialized skills that make them even more effective at specific tasks and domains.

#### Skills

Skills represent specialized capabilities that agents can develop and refine to excel in specific domains or tasks. Think of skills as the difference between a general practitioner and a specialist—while both are knowledgeable, the specialist has developed deep expertise, refined techniques, and domain-specific patterns that make them exceptionally effective in their area of focus.

In agentic systems, skills are implemented as structured combinations of prompts, tools, memory patterns, and domain knowledge that work together to solve specific types of problems. A financial analysis skill might combine market data tools, statistical calculation capabilities, and specialized prompts for interpreting economic indicators. A creative writing skill could integrate research tools, style guidelines, and iterative refinement processes.

Skills provide several key benefits: **Specialization** allows agents to develop deep expertise in specific areas rather than being mediocre generalists. **Consistency** ensures that similar problems are approached with proven, refined techniques. **Reusability** means successful skill patterns can be applied across different contexts and even shared between agents. **Composability** enables complex workflows where multiple skills collaborate to solve multifaceted problems.

However, skills also introduce challenges: they can create **over-specialization** where agents become inflexible, **complexity** that makes systems harder to debug and maintain, and **coordination overhead** when multiple skills need to work together. The key is finding the right balance between specialization and flexibility for your specific use case.

Let's implement a skill system that demonstrates these concepts:

In [ ]:
# Skills Implementation - Specialized agent capabilities
from dataclasses import dataclass
from typing import List, Dict, Any
from abc import ABC, abstractmethod

@dataclass
class SkillResult:
    """Result of executing a skill"""
    success: bool
    output: str
    confidence: float
    metadata: Dict[str, Any] = None

class BaseSkill(ABC):
    """Base class for agent skills"""
    
    def __init__(self, name: str, description: str):
        self.name = name
        self.description = description
        self.execution_count = 0
        
    @abstractmethod
    def execute(self, input_data: str, context: Dict[str, Any] = None) -> SkillResult:
        """Execute the skill with given input"""
        pass
    
    def get_metadata(self) -> Dict[str, Any]:
        """Get skill metadata and performance stats"""
        return {
            "name": self.name,
            "description": self.description, 
            "executions": self.execution_count
        }

# Financial Analysis Skill
class FinancialAnalysisSkill(BaseSkill):
    def __init__(self, llm):
        super().__init__(
            name="Financial Analysis",
            description="Analyze financial data and provide investment insights"
        )
        self.llm = llm
        self.analysis_prompt = PromptTemplate(
            input_variables=["data", "analysis_type"],
            template="""You are a senior financial analyst with expertise in investment analysis.
            
            Data to analyze: {data}
            Analysis type: {analysis_type}
            
            Provide a comprehensive analysis including:
            1. Key metrics interpretation
            2. Risk assessment
            3. Investment recommendation
            4. Confidence level (1-10)
            
            Focus on actionable insights and clearly explain your reasoning."""
        )
    
    def execute(self, input_data: str, context: Dict[str, Any] = None) -> SkillResult:
        self.execution_count += 1
        analysis_type = context.get("analysis_type", "general") if context else "general"
        
        try:
            chain = self.analysis_prompt | self.llm | StrOutputParser()
            result = chain.invoke({
                "data": input_data,
                "analysis_type": analysis_type
            })
            
            # Extract confidence from result (simplified)
            confidence = 0.8  # Would normally parse this from LLM output
            
            return SkillResult(
                success=True,
                output=result,
                confidence=confidence,
                metadata={"analysis_type": analysis_type}
            )
        except Exception as e:
            return SkillResult(
                success=False,
                output=f"Analysis failed: {str(e)}",
                confidence=0.0
            )

# Code Review Skill  
class CodeReviewSkill(BaseSkill):
    def __init__(self, llm):
        super().__init__(
            name="Code Review", 
            description="Perform comprehensive code reviews with security and best practice focus"
        )
        self.llm = llm
        self.review_prompt = PromptTemplate(
            input_variables=["code", "language", "focus_areas"],
            template="""You are a senior software engineer performing a detailed code review.
            
            Code to review:
            ```{language}
            {code}
            ```
            
            Focus areas: {focus_areas}
            
            Provide a structured review covering:
            1. Code quality and readability
            2. Security vulnerabilities
            3. Performance considerations
            4. Best practice compliance
            5. Specific improvement suggestions
            
            Rate each area from 1-10 and provide actionable feedback."""
        )
    
    def execute(self, input_data: str, context: Dict[str, Any] = None) -> SkillResult:
        self.execution_count += 1
        language = context.get("language", "python") if context else "python"
        focus_areas = context.get("focus_areas", "security, performance, readability") if context else "security, performance, readability"
        
        try:
            chain = self.review_prompt | self.llm | StrOutputParser()
            result = chain.invoke({
                "code": input_data,
                "language": language,
                "focus_areas": focus_areas
            })
            
            confidence = 0.85  # Would extract from actual analysis
            
            return SkillResult(
                success=True,
                output=result,
                confidence=confidence,
                metadata={"language": language, "focus_areas": focus_areas}
            )
        except Exception as e:
            return SkillResult(
                success=False,
                output=f"Code review failed: {str(e)}",
                confidence=0.0
            )

print("=== Skills System Implemented ===")
print("Created specialized skills for financial analysis and code review")

In [ ]:
# Skills Manager - Orchestrates and coordinates multiple skills
class SkillsManager:
    def __init__(self, llm):
        self.skills: Dict[str, BaseSkill] = {}
        self.llm = llm
        self.skill_selection_prompt = PromptTemplate(
            input_variables=["user_request", "available_skills"],
            template="""You are a skill coordinator. Given a user request, determine which skill(s) would be most appropriate.
            
            User Request: {user_request}
            
            Available Skills: {available_skills}
            
            Respond with just the skill name that best matches the request, or "none" if no skill is suitable.
            Consider the task type and choose the most specialized skill available."""
        )
    
    def register_skill(self, skill: BaseSkill):
        """Register a new skill with the manager"""
        self.skills[skill.name] = skill
        print(f"Registered skill: {skill.name}")
    
    def select_skill(self, user_request: str) -> str:
        """Intelligently select the best skill for a given request"""
        available_skills = "\n".join([
            f"- {name}: {skill.description}" 
            for name, skill in self.skills.items()
        ])
        
        chain = self.skill_selection_prompt | self.llm | StrOutputParser()
        selected_skill = chain.invoke({
            "user_request": user_request,
            "available_skills": available_skills
        }).strip()
        
        return selected_skill if selected_skill in self.skills else None
    
    def execute_skill(self, skill_name: str, input_data: str, context: Dict[str, Any] = None) -> SkillResult:
        """Execute a specific skill"""
        if skill_name not in self.skills:
            return SkillResult(
                success=False,
                output=f"Skill '{skill_name}' not found",
                confidence=0.0
            )
        
        return self.skills[skill_name].execute(input_data, context)
    
    def auto_execute(self, user_request: str, context: Dict[str, Any] = None) -> SkillResult:
        """Automatically select and execute the best skill for a request"""
        selected_skill = self.select_skill(user_request)
        
        if not selected_skill:
            return SkillResult(
                success=False,
                output="No suitable skill found for this request",
                confidence=0.0
            )
        
        print(f"Selected skill: {selected_skill}")
        return self.execute_skill(selected_skill, user_request, context)
    
    def get_skills_summary(self) -> Dict[str, Any]:
        """Get summary of all registered skills"""
        return {
            name: skill.get_metadata() 
            for name, skill in self.skills.items()
        }

# Create skills manager and register our skills
skills_manager = SkillsManager(memory_llm)
skills_manager.register_skill(FinancialAnalysisSkill(memory_llm))
skills_manager.register_skill(CodeReviewSkill(memory_llm))

print("=== Skills Manager Created ===")
print("Skills system ready for intelligent task routing")

In [ ]:
# Test the skills system with different types of requests

print("=== Skills System Demonstration ===")

# Test 1: Financial Analysis Request
financial_request = """
I have the following financial data for a tech company:
- Revenue: $50M (up 25% YoY)
- Operating margin: 15%
- Cash flow: $8M positive
- Debt-to-equity ratio: 0.3
- P/E ratio: 22

Should I invest in this company?
"""

print("--- Financial Analysis Test ---")
financial_result = skills_manager.auto_execute(
    financial_request,
    context={"analysis_type": "investment_decision"}
)
print(f"Success: {financial_result.success}")
print(f"Confidence: {financial_result.confidence}")
print(f"Result preview: {financial_result.output[:200]}...")

# Test 2: Code Review Request  
code_request = """
Please review this Python function:

def process_user_data(user_input):
    data = eval(user_input)
    sql = f"SELECT * FROM users WHERE id = {data['user_id']}"
    cursor.execute(sql)
    return cursor.fetchall()

Is this code secure and well-written?
"""

print("\n--- Code Review Test ---")
code_result = skills_manager.auto_execute(
    code_request,
    context={"language": "python", "focus_areas": "security, best practices"}
)
print(f"Success: {code_result.success}")
print(f"Confidence: {code_result.confidence}")
print(f"Result preview: {code_result.output[:200]}...")

# Test 3: General Request (no specific skill)
general_request = "What's the weather like today?"

print("\n--- General Request Test ---")
general_result = skills_manager.auto_execute(general_request)
print(f"Success: {general_result.success}")
print(f"Result: {general_result.output}")

# Show skills performance summary
print("\n=== Skills Performance Summary ===")
summary = skills_manager.get_skills_summary()
for skill_name, metadata in summary.items():
    print(f"{skill_name}: {metadata['executions']} executions")
    print(f"  Description: {metadata['description']}")

### Workflows and Chains



##### LangGraph Parallel Execution


##### LangGraph Multi-Agent Patterns


##### LangChain Agent Executors


##### LlamaIndex AgentRunner

## RAG

### But Why RAG?

Talk about LLM system in general, while introducing agents, where those workflows lack the limit of llms in context and actions

### Finding the Data

#### Webscraping


##### LangChain WebBaseLoader


##### LangChain AsyncHtmlLoader


##### LangChain SitemapLoader


##### LangChain PlaywrightURLLoader


##### LlamaIndex SimpleWebPageReader



##### LlamaIndex BeautifulSoupWebReader

#### Document Loading


##### LangChain PyPDFLoader


##### LangChain UnstructuredFileLoader


##### LangChain CSVLoader


##### LangChain JSONLoader


##### LlamaIndex SimpleDirectoryReader


##### LlamaIndex PDFReader

### Preprocessing the documents

#### Splitting


##### LangChain RecursiveCharacterTextSplitter


##### LangChain TokenTextSplitter


##### LangChain MarkdownHeaderTextSplitter


##### LangChain PythonCodeTextSplitter


##### LlamaIndex SentenceSplitter


##### LlamaIndex SemanticSplitterNodeParser


##### LlamaIndex HierarchicalNodeParser

#### Chunking



##### LangChain SemanticChunker


##### LangChain ParentDocumentRetriever


##### LlamaIndex SimpleNodeParser


##### LlamaIndex SentenceWindowNodeParser

#### Embedding


##### LangChain OpenAIEmbeddings


##### LangChain HuggingFaceEmbeddings


##### LlamaIndex OpenAIEmbedding



##### LlamaIndex HuggingFaceEmbedding

### Storing Documents

#### Vector Databases


##### LangChain Chroma Integration


##### LangChain Pinecone Integration


##### LangChain FAISS Integration


##### LlamaIndex ChromaVectorStore



##### LlamaIndex PineconeVectorStore

#### Knowledge Graphs


##### LangGraph StateGraph


##### LangChain Neo4jGraph



##### LlamaIndex KnowledgeGraphIndex

#### SQL


##### LangChain SQLDatabase


##### LangChain SQLDatabaseChain



##### LlamaIndex SQLStructStoreIndex

### Retrieval Mechanisms

#### Vector search


##### LangChain VectorStoreRetriever


##### LangChain MultiVectorRetriever


##### LlamaIndex VectorIndexRetriever



##### LlamaIndex VectorIndexAutoRetriever

#### Tree Search

#### Node Search

#### Hybrid Search

##### LangChain EnsembleRetriever
##### LangChain BM25Retriever
##### LlamaIndex QueryFusionRetriever

##### LangChain ConditionalEdge


##### LangGraph Router Patterns


##### LlamaIndex RouterQueryEngine

### Evaluation

#### Faithfulness & Accuracy

#### RAGAS (RAG Assessment)



##### LangSmith + RAGAS Integration


##### LangChain Evaluation Chains

#### TruLens RAG Triad


#### Multi-Agent Metrics


#### Advanced Agentic Patterns

#### Human Evaluation


#### LLM-as-Judge



##### LangSmith Tracing


##### LangSmith Evaluation Datasets


##### LangSmith Custom Evaluators

## A Complete Agentic System



##### LangGraph Agent Architecture


##### LangChain Agent Types (ReAct, Plan-and-Execute)


##### LangSmith Agent Monitoring


##### LlamaIndex Multi-Agent Orchestrator

## Limitations & Variations

#### RAPTOR

#### Self-RAG

#### CRAG

#### Adaptive RAG

## Summary

## Citations